# Classification using Cyclic Boosting

First, install the  package and its dependencies

```sh
!pip install cyclic-boosting
```

In [1]:
import pandas as pd
import numpy as np

Let's load the test dataset from Blue-Yonder-OSS

In [7]:
df = pd.read_csv("./archive_1/day.csv")
df = df.rename(columns={'dteday': 'date'})
df["date"] = pd.to_datetime(df["date"])
df.to_csv("./bike_sharing.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: './day.csv'

In [3]:
df.head()

,instant,date,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


# Automated Machine Learning with Tornado
With tornado, you can automatically perform data preparation, feature property setting, hyperparameter tuning, model building, training, evaluation, and plotting!

In [4]:
from cyclic_boosting.tornado import Generator, Manager, Trainer

data_deliverler = Generator.TornadoDataModule("./bike_sharing.csv")
manager = Manager.TornadoVariableSelectionModule()
trainer = Trainer.SqueezeTrainer(data_deliverler, manager)
trainer.run(target="cnt", log_policy="compute_COD", verbose=False)

Auto analysis target ['temp', 'atemp', 'hum', 'windspeed']


/home/tsuruda_yoshito/work/tornado/rPrivate-cyclic-boosting/.venv/lib/python3.10/site-packages/statsmodels/tsa/stl/mstl.py:217: UserWarning: A period(s) is larger than half the length of time series. Removing these period(s).
  warnings.warn(


    has_trend: ['temp', 'atemp', 'windspeed']
    has_seasonality: []
    has_up_monotonicity: []
    has_down_monotonicity: []
    has_linearity: []
    has_missing: []


iter: 85 / 135 Encountered negative change of loss. This might not be a problem, as long as the model converges. Check the LOSS changes in the analysis plots.
iter: 135 / 135 
TRUNCATED
['atemp', 'temp', 'mnth', 'season', 'yr', 'casual', 'weathersit', 'hum', 'windspeed', 'registered', 'holiday', 'workingday', 'instant', 'weekday', 'dayofweek', 'dayofyear', ('yr', 'atemp'), ('yr', 'temp'), ('yr', 'mnth'), ('season', 'yr'), ('weathersit', 'atemp'), ('weathersit', 'temp'), ('season', 'weathersit'), ('mnth', 'weathersit'), ('workingday', 'atemp'), ('workingday', 'temp'), ('season', 'atemp'), ('holiday', 'atemp'), ('yr', 'weathersit'), ('season', 'temp'), ('season', 'hum'), ('weekday', 'temp'), ('season', 'workingday'), ('mnth', 'holiday'), ('mnth', 'workingday'), ('temp', 'atemp'), ('mnth', 'atemp'), ('mnth', 'temp'), ('season', 'holiday'), ('yr', 'windspeed'), ('yr', 'workingday'), ('yr', 'holiday'), ('season', 'weekday'), ('season', 'dayofweek'), ('season', 'mnth'), ('yr', 'weekday'), ('

# Load the best model and make predictions.

Get the best model path.

In [5]:
import pickle
from pathlib import Path

model_nos = []
for p in sorted(Path("./models/").glob("model*")):
    model_nos.append(str(p)[str(p).find("_") + 1 :])
model_path = f"./models/model_{model_nos[-1]}/model_{model_nos[-1]}.pkl"
print(model_path)

./models/model_80/model_80.pkl


Make predictions with the best model.

In [6]:
data = {
    "dayofweek": [4],
    "dayofyear": [190],
    "event": [0],
    "l_id": [1],
    "normal_price": [10.34],
    "p_id": [20],
    "pg_id_1": [1],
    "pg_id_2": [1],
    "pg_id_3": [1],
    "promotion_type": [1],
    "sales_area": [6321.6800893695445],
    "sales_price": [10.34],
    "school_holiday": [0],
}

X = pd.DataFrame(data)

with open(model_path, "rb") as f:
    CB_est = pickle.load(f)
    yhat = CB_est.predict(X.copy())
    print(yhat)

ValueError: Input Matrix X has not the same number of feature columns (2) as the matrix in the fit (16).